In [3]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic"

## 1. GSEA analysis for each set of comparison

In [9]:
##########----- Parameters-----##########
analysis_name <- c("0", "resampled")
genome <- "mm"
#########################################
analysis_subdir <- paste(analysis_name[1], "Scanpy_out", analysis_name[2], sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "4_DE_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

###----- Find all de files
de.out.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "2_DE")
de.files <- list.files(de.out.dir, pattern="*differential.csv", full.names=TRUE,recursive=TRUE)

use.cols <- c("gene_names", "t-test_overestim_var_logfc", "t-test_overestim_var_padj")

###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
#gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv') #, gs.files
#gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/D6-5_IL2_dependent_mm_sigs.csv')
gs.files <- c('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4-ifn_mm_sigs.csv')
fc.cutoff <- 1

In [10]:
# For Arm v.s. Cl13: redo analysis and reduce fc cutoff
if (FALSE) {
    fc.cutoff <- 0
    sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "4_DE_GSEA_logfc0")
    dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
    setwd(sum.dir)

    de.out.dir.arm.cl13 <- file.path(base_dir, '1_Scanpy', analysis_subdir, "2_DE/perCluster_Arm_vs_Cl13")
    de.files <- list.files(de.out.dir.arm.cl13, pattern="*differential.csv", full.names=TRUE,recursive=TRUE)

    gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv',
                  '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/2020_mouse_CD8_mm_sigs.csv',
                  '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv',
                  '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD8_mm_sigs.csv')
}

In [11]:
completed.df <- data.frame()
for (gs.file.i in c("file", gs.files)) {
    gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
    completed.df[gs.file.base] <- character(0)
}
completed.name <- file.path(sum.dir, "completed.csv")

for (file.i in de.files) {
    file.i.qc.row <- c(file.i)
    
    ###----- Create output names & output directory
    file.i.outpath <- gsub(de.out.dir, "", file.i)
    file.i.outpath <- gsub(".csv", "", file.i.outpath)
    file.i.outpath <- gsub("^/", "",file.i.outpath)

    file.i.simp.name <- tail(unlist(strsplit( file.i.outpath, "/")), 1)
    file.i.simp.name <- gsub("_differential", "", file.i.simp.name)
    file.i.simp.vec <- unlist(strsplit(file.i.simp.name, "_vs_"))

    # Create output directory
    file.i.dir <- file.path(sum.dir, paste(file.i.outpath, "_sep", sep=""))
    dir.create(file.i.dir, recursive=TRUE, showWarnings=FALSE)
    
    for (gs.file.i in gs.files) {
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
        file.i.gs.dir <- file.path(file.i.dir, gs.file.base)
        dir.create(file.i.gs.dir, recursive=TRUE, showWarnings=FALSE)
        setwd(file.i.gs.dir)
        
        # Run GSEA analysis
        useGroup <- "t-test_overestim_var_logfc"
        outName <- file.i.simp.name
        
        success <- "yes"
        gsea <- tryCatch({
            GSEA_analysis(file.i, useGroup, outName, gs.file.i, fc.cutoff)
        }, error=function(cond) {
            message(paste(file.i, gs.file.i,sep='; '))
            return("no")
        })
        
        if (!is.null(gsea)){
            success <- ""
        }
        
        file.i.qc.row <- c(file.i.qc.row, success)
    }
    
    completed.df[nrow(completed.df) +1, ] <- file.i.qc.row
    write.csv(completed.df, completed.name, row.names=TRUE)
}


── Column specification ────────────────────────────────────────────
cols(
  gene_names = col_character(),
  A = col_double(),
  C = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 2577



── Column specification ────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (0.7% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ────────────────────────────────────────────
cols(
  gene_names = col_character(),
  A = col_double(),
  N = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `

[1] 6568



── Column specification ────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (0.92% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ────────────────────────────────────────────
cols(
  gene_names = col_character(),
  A = col_double(),
  C = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: 

...


── Column specification ────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (0.48% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ────────────────────────────────────────────
cols(
  gene_names = col_character(),
  A = col_double(),
  C = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: 

[1] 3978



── Column specification ────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (1.32% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ────────────────────────────────────────────
cols(
  gene_names = col_character(),
  A = col_double(),
  C = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: 

[1] 3978



── Column specification ────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (1.32% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...



## 2. GSEA analysis summary

### 2.1 Cl13 v.s. Arm

In [5]:
wk.dir <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'
setwd(wk.dir)

gsea.files.base <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'
a_c <- file.path(gsea.files.base, "A/A_vs_C_differential/A_vs_C---Combined_GEO_GSEA_signatures.csv")
a_n <- file.path(gsea.files.base, "A/A_vs_N_differential/A_vs_N---Combined_GEO_GSEA_signatures.csv")
c_a <- file.path(gsea.files.base, "C/C_vs_A_differential/C_vs_A---Combined_GEO_GSEA_signatures.csv")
c_n <- file.path(gsea.files.base, "C/C_vs_N_differential/C_vs_N---Combined_GEO_GSEA_signatures.csv")
n_a <- file.path(gsea.files.base, "N/N_vs_A_differential/N_vs_A---Combined_GEO_GSEA_signatures.csv")
n_c <- file.path(gsea.files.base, "N/N_vs_C_differential/N_vs_C---Combined_GEO_GSEA_signatures.csv")

gsea.files <- c(a_c, c_a, a_n, n_a, c_n, n_c)
gsea.sum.name <- "Arm-Cl13_gsea"


comp.order <- c("A_vs_C", "C_vs_A", "A_vs_N", "N_vs_A", "C_vs_N", "N_vs_C")

gsea.use.order <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/GSEA/Combined_GEO_GSEA_gs_order.csv'
gsea.use.order.tb <- read_csv(gsea.use.order)
path.order <- gsea.use.order.tb$gs_name

GSEA_sum(gsea.files, gsea.sum.name, 50, 15, TRUE, comp.order, TRUE, path.order)


── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character()
)




ERROR: Error in GSEA_sum(gsea.files, gsea.sum.name, 50, 15, TRUE, comp.order, : argument "filesimp_use_selfdefined" is missing, with no default


### 2.2 Time points

In [ ]:
wk.dir <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'
setwd(wk.dir)

gsea.files.base <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'

a5t_nav <- file.path(gsea.files.base, "A5T/A5T_vs_NP14B_differential/A5T_vs_NP14B---Combined_GEO_GSEA_signatures.csv")
a5p_nav <- file.path(gsea.files.base, "A5P/A5P_vs_NP14B_differential/A5P_vs_NP14B---Combined_GEO_GSEA_signatures.csv")
c5p_nav <- file.path(gsea.files.base, "C5P/C5P_vs_NP14B_differential/C5P_vs_NP14B---Combined_GEO_GSEA_signatures.csv")

a8t_a5t <- file.path(gsea.files.base, "A8T/A8T_vs_A5T_differential/A8T_vs_A5T---Combined_GEO_GSEA_signatures.csv")
a8p_a5p <- file.path(gsea.files.base, "A8P/A8P_vs_A5P_differential/A8P_vs_A5P---Combined_GEO_GSEA_signatures.csv")
c8p_c5p <- file.path(gsea.files.base, "C8P/C8P_vs_C5P_differential/C8P_vs_C5P---Combined_GEO_GSEA_signatures.csv")

c5p_a5p <- file.path(gsea.files.base, "C5P/C5P_vs_A5P_differential/C5P_vs_A5P---Combined_GEO_GSEA_signatures.csv")
c8p_a8p <- file.path(gsea.files.base, "C8P/C8P_vs_A8P_differential/C8P_vs_A8P---Combined_GEO_GSEA_signatures.csv")



gsea.files <- c(a5t_nav, a5p_nav, c5p_nav,
               a8t_a5t, a8p_a5p, c8p_c5p,
                c5p_a5p, c8p_a8p)

gsea.sum.name <- "Timepoint_Arm-Cl13-NAV_gsea"


comp.order <- c("A5T_vs_NP14B", "A5P_vs_NP14B", "C5P_vs_NP14B",
               "A8T_vs_A5T", "A8P_vs_A5P", "C8P_vs_C5P",
               "C5P_vs_A5P", "C8P_vs_A8P")

gsea.use.order <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/GSEA/Combined_GEO_GSEA_gs_order.csv'
gsea.use.order.tb <- read_csv(gsea.use.order)
path.order <- gsea.use.order.tb$gs_name

GSEA_sum(gsea.files, gsea.sum.name, 50, 18, TRUE, comp.order, TRUE, path.order)

### 2.3 Per cluster Cl13 v.s. Arm

In [ ]:
wk.dir <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'
setwd(wk.dir)

gsea.files.base <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/1_1_SCANPY_PAGA/all--numSlt-rmWTNAV/4_DE_GSEA/Arm_vs_Cl13'

gsea.files <- paste("P", c(1,2,3,5,6,7,8), sep="")
gsea.files <- file.path(gsea.files.base,
                        paste(gsea.files, "Arm_vs_cl13", sep="_"),
                        paste(gsea.files, "C", "vs",gsea.files, "A", "differential", sep="_"),
                        paste(gsea.files, "C", "vs", gsea.files, "A---Combined_GEO_GSEA_signatures.csv", sep="_"))

gsea.sum.name <- "Cluster_Arm-Cl13-NAV_gsea"


comp.order <- paste("P", c(1,2,3,5,6,7,8), sep="")
comp.order <- paste(comp.order, "C", "vs", comp.order, "A", sep="_")

gsea.use.order <- '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/GSEA/Combined_GEO_GSEA_gs_order.csv'
gsea.use.order.tb <- read_csv(gsea.use.order)
path.order <- gsea.use.order.tb$gs_name

GSEA_sum(gsea.files, gsea.sum.name, 50, 15, TRUE, comp.order, TRUE, path.order)